In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tensorflow.keras import layers

In [31]:
df_fake = pd.read_csv("Fake_News.csv")
df_fake

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [32]:
df_fake["target"] = 0
df_fake.head()

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [33]:
df_true = pd.read_csv("True_News.csv")
df_true

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [34]:
df_true["target"] = 1
df_true.head()

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [35]:
len(df_fake), len(df_true)

(23481, 21417)

In [36]:
df_fake_train = df_fake[:20000]
df_fake_test = df_fake[20000:]
df_true_train = df_true[:20000]
df_true_test = df_true[20000:]

In [37]:
df_train = pd.concat([df_fake_train, df_true_train], ignore_index=True)
df_train

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
39995,UK PM May will chair meeting of emergency comm...,LONDON (Reuters) - Prime Minister Theresa May ...,worldnews,"September 15, 2017",1
39996,Pakistan's top court rejects challenges to rem...,ISLAMABAD (Reuters) - Pakistan s Supreme Court...,worldnews,"September 15, 2017",1
39997,British police say 'terrorist incident' at Lon...,LONDON (Reuters) - British police said on Frid...,worldnews,"September 15, 2017",1
39998,Risk of Afghan civilian casualties could damp ...,KABUL (Reuters) - U.S. President Donald Trump ...,worldnews,"September 15, 2017",1


In [38]:
df_test = pd.concat([df_fake_test, df_true_test], ignore_index=True)
df_test

,title,text,subject,date,target
0,FEMALE SAILOR FACES DISCIPLINE By US Navy For ...,"***WARNING***If you watch this video, you will...",left-news,"Sep 9, 2016",0
1,ARROGANT BILL CLINTON MOCKS COAL MINERS For Su...,The arrogance of Bill Clinton is astounding. W...,left-news,"Sep 9, 2016",0
2,CHIP OFF THE OLD BLOCK: Harvard Bound Malia Ob...,Malia Obama is taking a party gap year before ...,left-news,"Sep 9, 2016",0
3,CRAZY VIDEO OF MUSLIM JIHADESS Being Dragged U...,Feel good video of the day A female ISIS recru...,left-news,"Sep 9, 2016",0
4,IDENTITY OF HILLARY’S MYSTERY “HANDLER” Is Fin...,"About a month ago, people started noticing a l...",left-news,"Sep 9, 2016",0
...,...,...,...,...,...
4893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
4894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
4895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
4896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


In [39]:
df_train.isna().sum()

,0
title,0
text,0
subject,0
date,0
target,0


In [40]:
df_train = df_train.dropna(axis=0)
df_train

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
39995,UK PM May will chair meeting of emergency comm...,LONDON (Reuters) - Prime Minister Theresa May ...,worldnews,"September 15, 2017",1
39996,Pakistan's top court rejects challenges to rem...,ISLAMABAD (Reuters) - Pakistan s Supreme Court...,worldnews,"September 15, 2017",1
39997,British police say 'terrorist incident' at Lon...,LONDON (Reuters) - British police said on Frid...,worldnews,"September 15, 2017",1
39998,Risk of Afghan civilian casualties could damp ...,KABUL (Reuters) - U.S. President Donald Trump ...,worldnews,"September 15, 2017",1


In [41]:
df_train.isna().sum()

,0
title,0
text,0
subject,0
date,0
target,0


In [42]:
df_train_shuffled = df_train.sample(frac=1)
df_train_shuffled.head()

,title,text,subject,date,target
37164,China vows to scrap secret interrogations of C...,BEIJING (Reuters) - China s ruling Communist P...,worldnews,"October 18, 2017",1
5033,"Reagan’s Son TEARS The ‘Absurdity’ Of Trump, ...","Ron Reagan, the late President Reagan s son, i...",News,"August 16, 2016",0
10707,"#LondonBridge MUSLIM Men CAUGHT Laughing, Cele...",Just another day in the holy month of Ramadan ...,politics,"Jun 3, 2017",0
2341,House GOP Puts DISGUSTING Restriction On Who’...,Paul Ryan said that the House would implement ...,News,"March 2, 2017",0
5946,Elizabeth Warren Just Proved Why She’s Trump’...,If there s anyone who has clearly had enough o...,News,"June 9, 2016",0


In [43]:
df_test_shuffled = df_test.sample(frac=1)
df_test_shuffled.head()

,title,text,subject,date,target
1496,SECRET DUMPS OF TOXIC WASTE ON PRIVATE PROPERT...,This story gets more and more unbelievable eve...,left-news,"Aug 17, 2015",0
126,BUSTED: HILLARY’S MEDIA CHEERLEADERS Caught Hi...,Wow! This compilation of blatant blocking for ...,left-news,"Aug 14, 2016",0
2085,The New Cold War: A Chilling Prospect for the ...,21st Century Wire says There is one curious th...,US_News,"June 24, 2017",0
3609,Coal mine explosion kills nine workers in nort...,BEIJING (Reuters) - An explosion late on Wedne...,worldnews,"September 14, 2017",1
4629,Colombia's FARC political party looks to coali...,BOGOTA (Reuters) - Colombia s disarmed FARC re...,worldnews,"September 1, 2017",1


In [44]:
random_samples = random.randint(0, len(df_train_shuffled)-5)
for row in df_train_shuffled[["title", "text", "target"]][random_samples:random_samples+5].itertuples():
  _, title, text, target = row
  print(f"\nTarget : {target}\nTitle : {title}\nText : {text}")
  print("="*20)


Target : 1
Title : Highlights: The Trump presidency on March 2 at 5:35 p.m. ET
Text : (Reuters) - Highlights of the day for U.S. President Donald Trump’s administration on Thursday: U.S. Attorney General Jeff Sessions removes himself from any investigations into alleged Russian meddling in the 2016 U.S. presidential election because he was involved with Trump’s campaign. The Russian Embassy to the United States says it had regular contact with “U.S. partners” after the Washington Post reported that Sessions had failed to disclose meetings with Russia’s ambassador. FBI Director James Comey is tight-lipped when asked about investigations into any Russian meddling in the U.S. election during a closed-door meeting in Congress, the leading Democrat on the House intelligence committee says. Former neurosurgeon Ben Carson wins Senate approval to become secretary of the Department of Housing and Urban Development, making him the sole African-American member of Trump’s Cabinet. The Senate vote

In [45]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(df_train_shuffled["text"].to_numpy(),
                                                                            df_train_shuffled["target"].to_numpy(),
                                                                            test_size=0.2,
                                                                            random_state=42)
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(32000, 8000, 32000, 8000)

In [46]:
max_vocab_length = 10000
max_length = 300

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [47]:
train_sentences[:5]

array(['How this didn t happen sooner, I do not know. But on Sunday, Ted Nugent appeared at a campaign rally with Donald Trump and it went just about how you d expect. Nugent played a little guitar, spewed some profanity and groped himself on the stage.Taking the stage at a Trump rally in Michigan, Nugent struggled with his amp before pumping out the national Anthem in the style of Jimi Hendricks. Who decided to give the man a microphone, I don t know, but the controversial rockstar took the opportunity to rant about  shit kickers  and ass kickers. I ve got your blue state right here, baby,  Nugent yelled at one point while grabbing his crotch and giving it a squeeze. Black and blue! Trump came out to give his speech after Nugent s antics but didn t mention them at all. He did, however, brag that he doesn t need guitars at his rallies. Apparently, it escaped his attention that Ted effing Nugent had just spent the past half hour playing guitar at his rally. There s no guitar  to draw a 

In [48]:
train_sentences.dtype

dtype('O')

In [49]:
text_vectorizer.adapt(train_sentences)

In [50]:
vocabs = text_vectorizer.get_vocabulary()
vocabs = [str(w) for w in vocabs]
top_5_vocabs = vocabs[:5]
bottom_5_vocabs = vocabs[-5:]

print(f"Top Vocabs : {top_5_vocabs}\nBottom Vocabs : {bottom_5_vocabs}")

Top Vocabs : ['', '[UNK]', 'the', 'to', 'of']
Bottom Vocabs : ['groped', 'forcefully', 'fist', 'exert', 'ernst']


In [51]:
embedder = Embedding(input_dim=max_vocab_length,
                     output_dim=128,
                     input_length=max_length)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [52]:
one_sample = random.choice(train_sentences)
embedder(text_vectorizer([one_sample]))

<tf.Tensor: shape=(1, 300, 128), dtype=float32, numpy=
array([[[-0.02436769,  0.00586542, -0.03917937, ..., -0.04539827,
         -0.04476427,  0.01833573],
        [ 0.04756275, -0.00012098, -0.04158732, ..., -0.01070018,
         -0.04611815, -0.03600854],
        [-0.01134435, -0.0096127 , -0.01643104, ...,  0.04940196,
          0.01194588, -0.01963975],
        ...,
        [ 0.0063213 ,  0.03406138, -0.02070375, ...,  0.01203052,
         -0.01765798,  0.01812042],
        [ 0.04756275, -0.00012098, -0.04158732, ..., -0.01070018,
         -0.04611815, -0.03600854],
        [ 0.03258071,  0.03945308, -0.01009429, ..., -0.04811839,
          0.03118351,  0.00441444]]], dtype=float32)>

In [53]:
model_baseline = Pipeline([
    ("Tfidf", TfidfVectorizer()),
    ("classification", MultinomialNB())
])

model_baseline.fit(train_sentences, train_labels)

Pipeline(steps=[('Tfidf', TfidfVectorizer()),
                ('classification', MultinomialNB())])

In [54]:
baseline_score = model_baseline.score(val_sentences, val_labels)
baseline_score

0.922

In [55]:
baseline_pred = model_baseline.predict(val_sentences)
baseline_pred[:10]

array([0, 1, 0, 1, 1, 1, 0, 0, 1, 1])

In [56]:
def model_results(y_true, y_pred):
  model_score = accuracy_score(y_true, y_pred)
  model_precision, model_recall, model_f1, _, = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  results = {"Accuracy" : model_score,
             "Precision" : model_precision,
             "Recall" : model_recall,
             "F1-Score" : model_f1}
  return results

In [57]:
baseline_results = model_results(val_labels, baseline_pred)
baseline_results

{'Accuracy': 0.922,
 'Precision': 0.9229120272333609,
 'Recall': 0.922,
 'F1-Score': 0.9219782591125081}

# Architecture 1 (LSTM + Dense(64))

In [58]:
# LSTM Model 1
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedder(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_LSTM_1 = tf.keras.Model(inputs, outputs)

model_LSTM_1.compile(loss="binary_crossentropy",
                   optimizer=tf.keras.optimizers.Adam(),
                   metrics=["accuracy"])

history_LSTM_1 = model_LSTM_1.fit(train_sentences,
                                  train_labels,
                                  epochs=5,
                                  validation_data=(val_sentences, val_labels))

Epoch 1/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 235s 232ms/step - accuracy: 0.7936 - loss: 0.4249 - val_accuracy: 0.9314 - val_loss: 0.2300
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 257s 227ms/step - accuracy: 0.9280 - loss: 0.2216 - val_accuracy: 0.9636 - val_loss: 0.1283
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 231s 231ms/step - accuracy: 0.9703 - loss: 0.0959 - val_accuracy: 0.9923 - val_loss: 0.0347
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 262s 231ms/step - accuracy: 0.9968 - loss: 0.0152 - val_accuracy: 0.9974 - val_loss: 0.0115
Epoch 5/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 255s 225ms/step - accuracy: 0.9986 - loss: 0.0064 - val_accuracy: 0.9980 - val_loss: 0.0068


In [59]:
LSTM_1_pred_probs = model_LSTM_1.predict(val_sentences)
LSTM_1_pred_probs[:10]

250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 64ms/step


array([[1.2590363e-05],
       [9.9996066e-01],
       [9.0038357e-04],
       [9.9998975e-01],
       [9.9969077e-01],
       [9.9952537e-01],
       [2.2156190e-04],
       [8.1303954e-04],
       [9.9974948e-01],
       [9.9907207e-01]], dtype=float32)

In [99]:
LSTM_1_preds = tf.squeeze(tf.round(LSTM_1_pred_probs))
LSTM_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 0., 1., 1., 1., 0., 0., 1., 1.], dtype=float32)>

In [61]:
LSTM_1_results = model_results(y_true=val_labels, y_pred=LSTM_1_preds)
LSTM_1_results

{'Accuracy': 0.998,
 'Precision': 0.9980011520933195,
 'Recall': 0.998,
 'F1-Score': 0.9980000146263903}

# Architecture 2 (LSTM with activation (tanh) + Dense(64))

In [62]:
# LSTM Model 2
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedder(x)
x = layers.LSTM(64, activation="tanh")(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_LSTM_2 = tf.keras.Model(inputs, outputs)

model_LSTM_2.compile(loss="binary_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

history_LSTM_2 = model_LSTM_2.fit(train_sentences,
                                  train_labels,
                                  epochs=5,
                                  validation_data=(val_sentences, val_labels))

Epoch 1/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 235s 231ms/step - accuracy: 0.8648 - loss: 0.3147 - val_accuracy: 0.9743 - val_loss: 0.0743
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 264s 234ms/step - accuracy: 0.9885 - loss: 0.0428 - val_accuracy: 0.9961 - val_loss: 0.0148
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 268s 240ms/step - accuracy: 0.9986 - loss: 0.0075 - val_accuracy: 0.9964 - val_loss: 0.0139
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 236s 236ms/step - accuracy: 0.9998 - loss: 6.7015e-04 - val_accuracy: 0.9976 - val_loss: 0.0090
Epoch 5/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 258s 232ms/step - accuracy: 1.0000 - loss: 1.3318e-04 - val_accuracy: 0.9977 - val_loss: 0.0094


In [100]:
LSTM_2_pred_probs = model_LSTM_2.predict(val_sentences)
LSTM_2_pred_probs[:10]

250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 92ms/step


array([[2.8197379e-05],
       [9.9999720e-01],
       [1.7487463e-04],
       [9.9999917e-01],
       [9.9998403e-01],
       [9.9998665e-01],
       [3.0581108e-01],
       [2.5603234e-05],
       [9.9998838e-01],
       [9.9997866e-01]], dtype=float32)

In [101]:
LSTM_2_preds = tf.squeeze(tf.round(LSTM_2_pred_probs))
LSTM_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 0., 1., 1., 1., 0., 0., 1., 1.], dtype=float32)>

In [102]:
LSTM_2_results = model_results(y_true=val_labels, y_pred=LSTM_2_preds)
LSTM_2_results

{'Accuracy': 0.978875,
 'Precision': 0.979428253794817,
 'Recall': 0.978875,
 'F1-Score': 0.9788727273197462}

# Architecture 3 (LSTM + GlobalAveragePoooling1D)

In [66]:
# LSTM Model 3
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedder(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.GlobalAveragePooling1D(name="global_avg_pool_1D")(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_LSTM_3 = tf.keras.Model(inputs, outputs)

model_LSTM_3.compile(loss="binary_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

history_LSTM_3 = model_LSTM_3.fit(train_sentences,
                                  train_labels,
                                  epochs=5,
                                  validation_data=(val_sentences, val_labels))

Epoch 1/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 245s 240ms/step - accuracy: 0.9634 - loss: 0.1200 - val_accuracy: 0.9980 - val_loss: 0.0088
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 258s 237ms/step - accuracy: 0.9992 - loss: 0.0034 - val_accuracy: 0.9979 - val_loss: 0.0059
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 264s 239ms/step - accuracy: 0.9994 - loss: 0.0017 - val_accuracy: 0.9990 - val_loss: 0.0034
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 265s 242ms/step - accuracy: 0.9998 - loss: 9.6719e-04 - val_accuracy: 0.9990 - val_loss: 0.0053
Epoch 5/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 260s 241ms/step - accuracy: 0.9997 - loss: 7.3403e-04 - val_accuracy: 0.9990 - val_loss: 0.0050


In [103]:
LSTM_3_pred_probs = model_LSTM_3.predict(val_sentences)
LSTM_3_pred_probs[:10]

250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 69ms/step


array([[9.5923545e-09],
       [9.9999988e-01],
       [6.2658785e-08],
       [9.9999994e-01],
       [9.9999899e-01],
       [9.9999964e-01],
       [4.0977698e-06],
       [1.2278501e-08],
       [9.9999988e-01],
       [9.9999523e-01]], dtype=float32)

In [104]:
LSTM_3_preds = tf.squeeze(tf.round(LSTM_3_pred_probs))
LSTM_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 0., 1., 1., 1., 0., 0., 1., 1.], dtype=float32)>

In [105]:
LSTM_3_results = model_results(y_true=val_labels, y_pred=LSTM_3_preds)
LSTM_3_results

{'Accuracy': 0.999,
 'Precision': 0.999001138592226,
 'Recall': 0.999,
 'F1-Score': 0.9990000073131953}

# Choose Best Model
Based on training history, Model_LSTM_2 was a little overfitting. let's select model among baseline_model, model_LSTM_1 and model_LSTM_3

In [83]:
baseline_results

{'Accuracy': 0.922,
 'Precision': 0.9229120272333609,
 'Recall': 0.922,
 'F1-Score': 0.9219782591125081}

In [106]:
LSTM_1_results

{'Accuracy': 0.998,
 'Precision': 0.9980011520933195,
 'Recall': 0.998,
 'F1-Score': 0.9980000146263903}

In [84]:
LSTM_3_results

{'Accuracy': 0.999,
 'Precision': 0.999001138592226,
 'Recall': 0.999,
 'F1-Score': 0.9990000073131953}

Looks like model_LSTM_3 performed well than model_LSTM_1, so let's save model_LSTM_2 to keras

In [107]:
model_LSTM_3.save("Detection_Fake_or_True_News.keras")

In [108]:
model_LSTM_3.save("Detection_Fake_or_True_News.h5")

# Check performance loaded model to test data and synthetic data

first i'm gonna predict test data

In [74]:
loaded_model = tf.keras.models.load_model("Detection_Fake_or_True_News.keras")

In [75]:
loaded_model_pred_probs = loaded_model.predict(df_test_shuffled["text"].to_numpy())
loaded_model_pred_probs

154/154 ━━━━━━━━━━━━━━━━━━━━ 10s 65ms/step


array([[1.6426413e-05],
       [4.0551196e-05],
       [2.3384720e-05],
       ...,
       [1.5665870e-05],
       [1.0459848e-02],
       [4.0579638e-05]], dtype=float32)

In [109]:
loaded_model_preds = tf.squeeze(tf.round(loaded_model_pred_probs))
loaded_model_preds

<tf.Tensor: shape=(4898,), dtype=float32, numpy=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)>

In [110]:
loaded_model_results = model_results(df_test_shuffled["target"].to_numpy(), loaded_model_preds)
loaded_model_results

{'Accuracy': 0.981625153123724,
 'Precision': 0.9818452577499318,
 'Recall': 0.981625153123724,
 'F1-Score': 0.9816853103758023}

let's testing model_LSTM_3 loaded version to predict synthetic data

In [115]:
true_sample = "NASA has successfully landed its Perseverance rover on the surface of Mars on February 18, 2021. The rover touched down in the Jezero Crater and will begin its mission to search for signs of ancient life. According to NASA, this mission is part of a broader effort to explore the Red Planet and prepare for future human missions."
fake_sample = "A former NASA employee has allegedly leaked documents proving the existence of alien life on Mars. According to anonymous sources, the Perseverance rover captured images of alien structures, but NASA has chosen to hide this from the public. Some experts believe that this is part of a global conspiracy to control information about extraterrestrial life."

In [116]:
synthetic_pred_probs = loaded_model.predict(tf.constant([true_sample, fake_sample]))
synthetic_pred_probs

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


array([[9.988957e-01],
       [8.278665e-04]], dtype=float32)

In [117]:
synthetic_preds = tf.squeeze(tf.round(synthetic_pred_probs))
synthetic_preds

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 0.], dtype=float32)>

In [118]:
class_names = ["Fake News", "True News"]

In [120]:
print(f"The News is :\n{true_sample}\nPredicted as : {class_names[int(synthetic_preds[0])]}\nWhere the Prediction Probability as True News is : {synthetic_pred_probs[0].item()*100:.2f}%\n")
print(f"The News is :\n{fake_sample}\nPredicted as : {class_names[int(synthetic_preds[1])]}\nWhere the Prediction Probability as True News is : {synthetic_pred_probs[1].item()*100:.2f}%")

The News is :
NASA has successfully landed its Perseverance rover on the surface of Mars on February 18, 2021. The rover touched down in the Jezero Crater and will begin its mission to search for signs of ancient life. According to NASA, this mission is part of a broader effort to explore the Red Planet and prepare for future human missions.
Predicted as : True News
Where the Prediction Probability as True News is : 99.89%

The News is :
A former NASA employee has allegedly leaked documents proving the existence of alien life on Mars. According to anonymous sources, the Perseverance rover captured images of alien structures, but NASA has chosen to hide this from the public. Some experts believe that this is part of a global conspiracy to control information about extraterrestrial life.
Predicted as : Fake News
Where the Prediction Probability as True News is : 0.08%
